In [34]:
import os
from netCDF4 import Dataset
import numpy as np
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import datetime

In [35]:
directory_path = "/Users/michaelvaden/Downloads/SST_Files/"

data = []
simulations = []

# add logic to sort files first by sim and year
file_list = sorted([filename for filename in os.listdir(directory_path) if filename.endswith(".nc")])

# Iterate through each file in the directory
for filename in file_list:
    file_path = os.path.join(directory_path, filename)

    file = Dataset(file_path, 'r')
    
    ds = xr.open_dataset(file_path, decode_times=False)

    reference_date = pd.to_datetime("1850-01-01")
    start_date = reference_date + datetime.timedelta(days = float(ds.time[0]))
    ds['time'] = pd.date_range(start=start_date, periods=ds.sizes['time'], freq='MS')

    # Select Jul-Oct
    ds = ds.sel(time=ds['time.month'].isin([7, 8, 9, 10])).resample(time='AS').mean()

    # Get rid of 2014 data
    mask = ds['time'].dt.year != 2014
    ds = ds.sel(time=mask)

    simulations.append([ds.attrs['case'][-8:]]*len(ds['time']))

    data.append(ds['SST'])

In [36]:
combined_sst = xr.concat(data, dim='time')

combined_sst

<xarray.DataArray 'SST' (time: 7400, lat: 192, lon: 288)>
array([[[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.44324, 271.44354, 271.44382, ..., 271.4425 , 271.44275,
         271.443  ],
        [271.4478 , 271.4479 , 271.44806, ..., 271.44745, 271.44757,
         271.44766],
        [271.4512 , 271.4512 , 271.4512 , ..., 271.4512 , 271.4512 ,
         271.4512 ]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
...
        [271.5104 , 271.5103 , 271.5102 , ..., 271.51056, 271.5105 ,
         271.51047],
        [271.51163, 271.51154, 271.51154, ..., 271.51178, 271.51172,
         271.51166],
        [271.51505, 271.515  , 271.515  , ..., 271.5151 , 271.51508,
         271.51505]],

       [[  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        [  0.     ,   0.     ,   0.     , ...,   0.     ,   0.     ,
           0.     ],
        ...,
        [271.59625, 271.5949 , 271.59363, ..., 271.59998, 271.59875,
         271.59753],
        [271.59726, 271.59686, 271.5965 , ..., 271.5985 , 271.59808,
         271.59766],
        [271.60504, 271.60498, 271.60495, ..., 271.60513, 271.6051 ,
         271.60504]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 1940-01-01 1941-01-01 ... 2013-01-01
Attributes:
    units:         K
    long_name:     sea surface temperature
    cell_methods:  time: mean

In [37]:
print(len(simulations))

sims = 0
for s in simulations:
    for d in s:
        sims += 1

print(sims)
    

800
7400


74 years per sample x 1 average of 4 months per sample x 100 simulations = 7400

In [38]:
combined_sst.to_netcdf('/Users/michaelvaden/Downloads/SST_Data.nc')

Can open the file like this:

In [41]:
dstest = xr.open_dataset('/Users/michaelvaden/Downloads/SST_Data.nc', decode_times=True)

dstest

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, time: 7400)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 1940-01-01 1941-01-01 ... 2013-01-01
Data variables:
    SST      (time, lat, lon) float32 ...

And access data like this:

In [42]:
dstest['SST']

<xarray.DataArray 'SST' (time: 7400, lat: 192, lon: 288)>
[409190400 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 1940-01-01 1941-01-01 ... 2013-01-01
Attributes:
    units:         K
    long_name:     sea surface temperature
    cell_methods:  time: mean